In [0]:


from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from matplotlib import pyplot as plt
import numpy as np
import keras
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.datasets import imdb
np.load.__defaults__= (None, True, True, 'ASCII')
import sys


from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve



from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 




np.set_printoptions(threshold=sys.maxsize)
%matplotlib inline



word_index = imdb.get_word_index()


word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])
  
 


(training_X, training_y), (testing_X, testing_y) = imdb.load_data(num_words=10000)
X_text = np.concatenate((training_X, testing_X), axis = 0)
data = np.concatenate((training_X, testing_X), axis=0)
targets = np.concatenate((training_y, testing_y), axis=0)


def vectorize(sequences, dimension = 10000):
  res = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    res[i, sequence] = 1
  return res
 
data = vectorize(data)
targets = np.array(targets).astype("float32")





X_test = data[:10000]
y_test = targets[:10000]
X_train = data[10000:]
y_train = targets[10000:]
model = models.Sequential()



model.add(layers.Dense(50, activation = "relu", input_shape=(10000,)))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()



#keras.optimizers.Adam(lr=0.1)
#keras.activations.relu(x, alpha=0.0, max_value=None, )
def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  
#plot_roc_curve (fpr,tpr) 
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)




# Обучение первоначальной модели
history = model.fit(
 X_train, y_train,
 epochs= 2,
 batch_size = 400,
 validation_data = (X_test, y_test)
)



#Визуализация матрицы невязки по двум классам 
predicted = model.predict(data, verbose=1)
#predicted = np.transpose(predicted)
predict = np.array([])
predict = [0] * len(data)
error_matrix  = np.array([])
error_matrix = [0] * len(data)

for x in np.arange(len(predicted)):
   if predicted[x] >= 0.5:
      predict[x] = 1.
   elif predicted[x] < 0.5:
      predict[x] = 0.
predict = np.transpose(predict)

for x in np.arange(len(targets)):
  if(targets[x] == predict[x]):
    error_matrix[x] = 1.
  else:
    error_matrix[x] = 0.
error_matrix = np.transpose(error_matrix)
print("Матрица невязки по двум классам:")    
print(error_matrix)    


# Построение ROC кривой
y_pred_keras = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

auc_keras = auc(fpr_keras, tpr_keras)

rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf)
auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()


#Вывод ревью, на которых лучшая модель ошиблась
print("Вывод ревью, на которых лучшая модель ошиблась:")
for i in np.arange(10000):
  if error_matrix[i] == 0.:
      print(decode_review(X_text[i]))


print("Test-Accuracy:", np.mean(history.history["val_acc"]))
#print("Test-Accuracy:", np.mean(results.history[keras_metrics.precision(), keras_metrics.recall()]))



Судя по отзывам на которых ошиблась модель, ошибка могла произойти из большого числа
неизвестных слов и малого количества слов прилагательных, которые описывают фильм.


С кодом всё понятно, но не мог бы ты визуализировать результаты обучения?

1) Оформи все в Jupyter Notebook.

2) В ноутбуке сохрани записи о точности модели на валидационной выборке в зависимости от количества пройденных эпох, параметра learning rate в оптимизаторе, трешхолда (по умолчанию трешхолд = 0.5, то есть если предсказание от 0 до 0.5 - ревью отрицательное, если от 0.5 до 1 - положительное).

3) Визуализируй матрицу невязки по 2 классам (положительное/негативное ревью), посчитай метрики precision/recall для отрицательных ревью, построй график ROC-кривой.

4) Выведи ревью, на которых лучшая модель ошиблась, сделай предположение, почему могла произойти ошибка.

5) Попробуй усложнить архитектуру, используй LSTM слои (примеры реализации можно найти в интернете даже по этой же задаче), сравни точность с начальной моделью.



In [0]:
# Модель для подсчитывания precision и recall для отрицательных отзывов
import numpy as np
import keras_metrics
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb

np.load.__defaults__=(None, True, True, 'ASCII')

(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)
def vectorize(sequences, dimension = 10000):
 results = np.zeros((len(sequences), dimension))
 for i, sequence in enumerate(sequences):
  results[i, sequence] = 1
 return results

data = vectorize(data)
targets = np.array(targets).astype("float32")




i = 0
# Создаем матрицы отрицательных ревью



X_negative = np.zeros((50000, 10000))
X_negative = np.array(X_negative).astype("float32")


# Цикл очень долго работает, а другого пути отобрать негативные отзывы я не нашел
for i in np.arange(50000):
  if targets[i] == 0.:
     #np.append(X_negative, data[i])
     np.append(X_negative, data[i])


     
y_negative = np.zeros(25000)

print(data[0:2])


print("************************")

print(X_negative[0:2])
sys.exit()


model = models.Sequential()

model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()

model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
 #metrics=[keras_metrics.precision(), keras_metrics.recall()]
)

history = model.fit(
 X_negative, y_negative,
 validation_split=0.20,
 epochs = 2,
 batch_size = 400 
)

print(history.history)
print("Test-Accuracy:", np.mean(results.history["val_acc"]))



In [0]:
# Модель с использованием LSTM слоев

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 10000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)



In [0]:
0.8937499892711639 -  0.8078



### При использовании слоев LSTM точность сети падает. Скорее всего для лучшего качества на LSTM необходимо больше данных. 

*25000/25000 [==============================] - 27s 1ms/step
Test score: 1.1325414541220664*
Test accuracy: **0.8078**

При использовании обычных слоев точность модели: **0.8937499892711639**

**Разница** =  0.8937499892711639 -  0.8078 = 0.08594998927116393


Точность так же падает при увеличении количества эпох на первой модели.

Точность не меняется при learning rate = 0.1, 0.01, 0.001.
